# Why Prompt Engineering

1. Model responses are stochastic: The same prompt will likely produce different responses with different models or model versions.
2. Models can fabricate responses: hellucication and fabrication.
3. Models capabilities will vary.

All what you need to run the code is have a **.env** file, with the following key
```bash
OPENAI_API_KEY=
```

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv()


True

In [2]:
client = OpenAI()

def get_completion(prompt, model ="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0,
    )
    return response.choices[0].message.content

# Types of prompts

## 1. Basic Prompt

In [3]:
prompt = f"""
Oh say can you see
"""
response = get_completion(prompt)
print (response)

By the dawn's early light


## 2. Complex prompt

In [4]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        { "role": "system", "content": "You are a helpful assistant."},
        { "role": "user", "content": "Who won the world series in 2020"},
        { "role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020." },
        { "role": "user", "content": "Where was it played?" }
    ]
)
print (response.choices[0].message.content)

The 2020 World Series was played at Globe Life Field in Arlington, Texas.


In [5]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        { "role": "system", "content": "You are a helpful assistant."},
        { "role": "user", "content": "Who won the world series in 2020"},
        { "role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020." },
        { "role": "user", "content": "Where was it played?" }
    ]
)
print (response.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The World Series in 2020 was played at a neutral site due to the COVID-19 pandemic. The games were played at Globe Life Field in Arlington, Texas.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]


# Principles

1. Write clear and specific instructions
2. Give the model time to "think"

To do that we use the following tacktics:

# Write clear and specifig instructions:

## 1. Use delimiters to clearly indicate distinct parts of the input

delimiters can be anything like:  
* triple quotes: """
* triple backticks: ```
* triple dashes: ---
* angle brackets: <>
* xml tags: <tag> </tag>

In [6]:
text = f"""
Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of the Sun, \
but two-and-a-half times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations \
since before recorded history. It is named after the Roman god Jupiter. When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows, and is on average the third-brightest natural object in the night sky after the Moon and Venus.
"""


In [7]:
prompt = f"""
Summarize the text delimited by triple backticks \
```{text}```
"""

In [8]:
response = get_completion(prompt)
print (response)

Jupiter is the largest planet in the Solar System, known to ancient civilizations and named after the Roman god Jupiter. It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all other planets combined. Jupiter is one of the brightest objects in the night sky, visible to the naked eye, and can cast visible shadows when viewed from Earth.


## 2. Ask for structured output

In [9]:
prompt = f"""
summarize the facts in the text delimited by triple backticks and print them in bullet list:
```{text}```
"""

In [10]:
response = get_completion(prompt)
print (response)

- Jupiter is the fifth planet from the Sun
- Jupiter is the largest planet in the Solar System
- Jupiter is a gas giant
- Jupiter has a mass one-thousandth that of the Sun
- Jupiter's mass is two-and-a-half times that of all the other planets in the Solar System combined
- Jupiter is one of the brightest objects visible to the naked eye in the night sky
- Jupiter has been known to ancient civilizations since before recorded history
- Jupiter is named after the Roman god Jupiter
- Jupiter can be bright enough for its reflected light to cast visible shadows when viewed from Earth
- Jupiter is on average the third-brightest natural object in the night sky after the Moon and Venus


## 3. Ask the model to check weather conditions are satisfied

In [11]:
prompt = f"""
you will be provided by text delimited by triple backticks.
if the text contains facts, put them in following format:
fact 1 - ...
fact 2 - ....
...

if the text contains the distance between Jupiter and Earh, then write at the end : \"distance from Earth:\", if not say \"the distance to Earth is unknown so far\".

```{text}```
"""


In [12]:
response = get_completion(prompt)
print (response)

fact 1 - Jupiter is the fifth planet from the Sun.
fact 2 - Jupiter is the largest planet in the Solar System.
fact 3 - Jupiter is a gas giant.
fact 4 - Jupiter has a mass one-thousandth that of the Sun.
fact 5 - Jupiter's mass is two-and-a-half times that of all the other planets in the Solar System combined.
fact 6 - Jupiter is one of the brightest objects visible to the naked eye in the night sky.
fact 7 - Jupiter has been known to ancient civilizations since before recorded history.
fact 8 - Jupiter is named after the Roman god Jupiter.
fact 9 - Jupiter can be bright enough for its reflected light to cast visible shadows when viewed from Earth.
fact 10 - Jupiter is on average the third-brightest natural object in the night sky after the Moon and Venus.

the distance to Earth is unknown so far.


## 4. "Few-shot" prompting

In [13]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest
valley flows from a modest spring; the 
grandest symphony originates from a single note; 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a mighty oak tree that withstands the fiercest storms, bending but never breaking. It is the ability to bounce back from adversity, to find strength in the face of challenges, and to keep moving forward despite setbacks. Just like the oak tree, resilience grows stronger with each trial it endures.


# Give the model time to think

## 1. Specify the steps required to complete a task

### example 1

In [16]:
text = f"""
In a charming village, siblings Jack and Jill set out on 
a quest to fetch water from a hilltop 
well. As they climbed, singing joyfully, misfortune
struck—Jack tripped on a stone and tumbled 
down the hill, with Jill following suit. 
Though slightly battered, the pair returned home to 
comforting embraces. Despite the mishap, 
their adventurous spirits remained undimmed, and they 
continued exploring with delight.
"""

prompt = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple 
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following 
keys: french_summary, num_names.

Separate your answers with line breaks.

Text: 
```{text}```
"""

In [17]:
response = get_completion(prompt)
print (response)

1 - Jack and Jill, siblings from a charming village, go on a quest to fetch water from a hilltop well, but encounter misfortune along the way.

2 - Jack et Jill, frère et sœur d'un charmant village, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent des malheurs en chemin.

3 - Jack, Jill

4 - 
{
  "french_summary": "Jack et Jill, frère et sœur d'un charmant village, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent des malheurs en chemin.",
  "num_names": 2
}


### 2. Ask for the output in a specified format

In [18]:
prompt = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt)
print("\nCompletion for prompt :")
print(response)


Completion for prompt :
Summary: Jack and Jill, two siblings, go on a quest to fetch water from a hilltop well but encounter misfortune along the way.

Translation: Jack et Jill, deux frères et sœurs, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin.

Names: Jack, Jill

Output JSON: 
{
  "french_summary": "Jack et Jill, deux frères et sœurs, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin.",
  "num_names": 2
}


### 3. Instruct the model to work out its own solution before rushing to a conclusion

In [20]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need 
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost 
me a flat $100k per year, and an additional $10 / square 
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet is indeed 450x + 100,000.


**Note that the student's solution is not correct**

**Now fix it by instructing the model to work out its own soloution first**

In [21]:
prompt = f"""
Your task is to determine if the student's solution 
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution 
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help 
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost 
me a flat $100k per year, and an additional $10 / square 
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 + $10 * x

Total cost for the first year of operations:
Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = $100x + $250x + $100,000 + $10x
Total cost = $360x + $100,000

So, the total cost for the first year of operations as a function of the number of square feet is 360x + 100,000.

Is the student's solution the same as the actual solution just calculated:
```
No
```
Student grade:
```
incorrect
```
